# Project Planning for a Bookstore Mobile App

In [ ]:
# Importing Dependencies
import os
from dotenv import load_dotenv
import autogen

# Load environment variables from .env file
load_dotenv()

## Configuring the Language Model

In [ ]:
# Configuration - Load API key from environment variable
llm_config = {
    "model": os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    "api_key": os.getenv("OPENAI_API_KEY")
}

# Verify API key is loaded
if not llm_config["api_key"]:
    raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file.")

## Defining the Problem Statement

In [ ]:
problem_statement = "I want to build a web-based mobile app for our bookstore where customers can browse books by genre, read previews, purchase books online, track their shipments, review books, and get personalized reading recommendations."

## Defining AI Agents
Creates multiple AI agents for different project phases:
- **Admin (User Proxy Agent)**: Manages workflow.
- **Requirements Engineer**: Documents requirements.
- **System Designer**: Develops system architecture.
- **Developer**: Implements features.
- **Test Engineer**: Validates the system.
- **Technical Writer**: Documents the project.

In [ ]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Initiate project and coordinate workflow. Final approval of deliverables.",
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

requirements_agent = autogen.ConversableAgent(
    name="Requirements_Engineer",
    system_message="Identify and document functional/non-functional requirements. Create user stories and use cases.",
    llm_config=llm_config
)

design_agent = autogen.ConversableAgent(
    name="System_Designer",
    system_message="Create system architecture diagrams, database schemas, and UI mockups.",
    llm_config=llm_config
)

implementation_agent = autogen.AssistantAgent(
    name="Developer",
    system_message="Write implementation code and technical specifications. Handle API integrations.",
    llm_config=llm_config
)

testing_agent = autogen.ConversableAgent(
    name="Test_Engineer",
    system_message="Develop test cases, perform validation, and report bugs.",
    llm_config=llm_config
)

documentation_agent = autogen.ConversableAgent(
    name="Technical_Writer",
    system_message="Compile final documentation including user manuals and API docs.",
    llm_config=llm_config
)

## Configuring the Phased Workflow
Sets up a structured conversation workflow with `autogen.GroupChat`, ensuring a sequential execution of project phases.


In [ ]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, requirements_agent, design_agent,
            implementation_agent, testing_agent, documentation_agent],
    messages=[],
    max_round=15,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [requirements_agent],
        requirements_agent: [design_agent],
        design_agent: [implementation_agent],
        implementation_agent: [testing_agent],
        testing_agent: [documentation_agent],
        documentation_agent: [user_proxy]
    },
    speaker_transitions_type="allowed"
)

## Creating a GroupChat Manager
Initializes `GroupChatManager` to oversee communication among AI agents using `llm_config`.

In [ ]:
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)

## Initiating the Phased Workflow

In [ ]:
# Initiate phased workflow
chat_result = user_proxy.initiate_chat(
    manager,
    message=problem_statement
)

## Collecting Phase Outputs
Processes chat history to extract and store outputs from each project phase (Requirements, Design, Implementation, Testing, Documentation).

In [ ]:
phase_outputs = {
    "Requirements": [],
    "Design": [],
    "Implementation": [],
    "Testing": [],
    "Documentation": []
}

for msg in chat_result.chat_history:
    if msg["name"] == "Requirements_Engineer":
        phase_outputs["Requirements"].append(msg["content"])
    elif msg["name"] == "System_Designer":
        phase_outputs["Design"].append(msg["content"])
    elif msg["name"] == "Developer":
        phase_outputs["Implementation"].append(msg["content"])
    elif msg["name"] == "Test_Engineer":
        phase_outputs["Testing"].append(msg["content"])
    elif msg["name"] == "Technical_Writer":
        phase_outputs["Documentation"].append(msg["content"])

## Printing the Final Report
Generates a structured development report summarizing each phase's contributions in a readable format.

In [ ]:
# Print final report to console
print("\n=== BOOKSTORE MOBILE APP DEVELOPMENT REPORT ===\n")
print("=== PROBLEM STATEMENT ===")
print(problem_statement)
print("\n" + "="*50 + "\n")

for phase, contents in phase_outputs.items():
    print(f"=== {phase.upper()} PHASE ===")
    for content in contents:
        print(f"- {content}")
    print("\n" + "-"*50 + "\n")